In [ ]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET
import pandas as pd
import re

In [ ]:
# Check if the directory with XML files exists, if not create it
xml_directory = Path('xml_files')
if not os.path.exists(xml_directory):
    os.makedirs(xml_directory)

output_directory = Path('excel_files')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Find all XML files in the directory
xml_files = [f for f in xml_directory.glob('*.xml')]
print(f"Found {len(xml_files)} xml-files in the '{xml_directory}' directory.")

# Take the first XML file for processing
first_xml_file = xml_files[0]
first_xml_file

In [ ]:
def extract_data_from_xml_file(xml_file):

    print('Parsing', xml_file.name)
    tree = ET.parse(xml_file)
    root = tree.getroot()

    def strip_tag(tag):
        return tag.split('}')[-1].lower() if tag else ''

    def parse_float(s):
        if s is None:
            return None
        s = str(s).strip()
        if s == '':
            return None
        s = s.replace(' ', '')  # non-breaking space
        s = s.replace(',', '.')
        s = re.sub(r'[^0-9.]', '', s)
        try:
            return float(s)
        except Exception:
            return None

    num_indicators = ['quantita','prezzo','prezzounitario','prezzototale','aliquota','percentuale','importo','valore','amount','price','quantity']
    lines = []
    columns = set()
    for elem in root.iter():
        if strip_tag(elem.tag) != 'dettagliolinee':
            continue
        data = {}
        # flatten one level: child with text -> key, child with subelements -> child_subkey
        for child in elem:
            tag = strip_tag(child.tag)
            # if child has subelements, flatten them
            if any(True for _ in child):
                for sub in child:
                    sk = strip_tag(sub.tag)
                    key = f'{tag}_{sk}'
                    val = sub.text.strip() if sub.text and sub.text.strip() else None
                    data[key] = val
                    columns.add(key)
            else:
                val = child.text.strip() if child.text and child.text.strip() else None
                data[tag] = val
                columns.add(tag)
        # convert obvious numeric fields
        for k, v in list(data.items()):
            kl = k.lower()
            if 'numero' in kl or kl == 'numerolinea':
                try:
                    data[k] = int(v) if v is not None else None
                except Exception:
                    pass
            elif any(ind in kl for ind in num_indicators):
                data[k] = parse_float(v)
        lines.append(data)

    # Ensure consistent column order: sort columns with numeric-like ones next to each other
    columns = list(columns)

    # Create DataFrame and reindex to include all discovered columns
    df = pd.DataFrame(lines)

    # Reorder columns: put numerolinea first if present
    if 'numerolinea' in df.columns:
        cols = ['numerolinea'] + [c for c in df.columns if c != 'numerolinea']
        df = df[cols]


        # Extract DatiGeneraliDocumento data
        dati_generali = {}
        for elem in root.iter():
            if strip_tag(elem.tag) == 'datigeneralidocumento':
                for child in elem:
                    tag = strip_tag(child.tag)
                    val = child.text.strip() if child.text and child.text.strip() else None
                    dati_generali[tag] = val
                break

        # Add DatiGeneraliDocumento fields to each line
        for line in lines:
            line.update(dati_generali)

        # Recreate DataFrame with the updated lines
        df = pd.DataFrame(lines)

        





        # Reorder columns: put DatiGeneraliDocumento fields first, then numerolinea
        dati_cols = list(dati_generali.keys())
        if 'numerolinea' in df.columns:
            cols = dati_cols + ['numerolinea'] + [c for c in df.columns if c not in dati_cols + ['numerolinea']]
        else:
            cols = dati_cols + [c for c in df.columns if c not in dati_cols]
        df = df[cols]

    print('Extracted lines:', len(df))
    print('Discovered columns:', list(df.columns))
    print()

    return df


In [ ]:
_ = extract_data_from_xml_file(first_xml_file)

In [ ]:
# Initialize an empty DataFrame to store merged data
merged_df = pd.DataFrame()

for xml_file in xml_files:
    df = extract_data_from_xml_file(xml_file)
    if df is not None and not df.empty:
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save merged DataFrame to Excel
output_file = output_directory / 'merged_invoices.xlsx'
merged_df.to_excel(output_file, index=False)